In [2]:
%load_ext autoreload
%autoreload 2

In [72]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import requests
import pandas as pd
import config
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from assignment1 import *
import re
import time
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
client = Socrata("data.cityofchicago.org",config.key)
conn = sqlite3.connect('db.db')
c = conn.cursor()
from pandas import read_sql_query
pq = lambda x: read_sql_query(x,conn)

## Scope of Analysis, Summer 2016
In the following analyses, we use data from the following datasets, focussing specifically on the months of June, July, and August of 2016:
- [311 Graffiti Removal Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Graffiti-Removal/hec5-y4x5)
- [311 Vacant and Abandoned Buildings Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Vacant-and-Abandoned-Building/7nii-7srd)
- [311 Alley Light Out Requests](https://data.cityofchicago.org/Service-Requests/311-Service-Requests-Alley-Lights-Out/t28b-ys7j)
- [American Community Survey 2016, US Census](https://www.census.gov/data/developers/data-sets/acs-5year.html)

## Initial Data Exploration

## Service Request Counts for Summer 2016
In June, July, and August of 2016, there were significantly more Graffiti removal requests compared to Alley Light Out and Vacant/Abandoned Buildings Reports. 

In [40]:
barplot(pq('''
SELECT type_of_service_request,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data
GROUP BY type_of_service_request
;'''),
x = 'type_of_service_request',
y = 'Count', 
xlabel = 'Service Request Type', 
ylabel ='Service Request Type', 
title = "Service Request Type Counts for Summer 2016")

## Service Requests by Month
There were more service requests in August 2016 across these three categories compared to June and July.

In [41]:
r = pq('''
SELECT type_of_service_request, strftime('%m',creation_date) as month,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data
GROUP BY type_of_service_request, month''')
trace1 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Alley Light Out']),
    name='Alley Light Out'
)
trace2 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Graffiti Removal']),
    name='Graffiti Removal'
)
trace3 = go.Bar(
    x=['June 2016', 'July 2016', 'August 2016'],
    y=list(r.Count[r.type_of_service_request == 'Vacant/Abandoned Building']),
    name='Vacant/Abandoned Building'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack',
    title='<b>Service Requests by Month</b><br>',
             xaxis=dict(
                title= 'Month',
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
             yaxis=dict(
                title='Service Request Count',
                titlefont=dict(
                    size=16,
                    color='rgb(107, 107, 107)'
                ),
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
            )

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

## Number of Service Requests by Community Area
West town's residents and patrons submit 400+ more graffiti, alley light, and vacant building requests during this timeframe. What follows is a breakdown by Service Request Type to see if the pattern persists.

In [42]:
barplot(pq('''
SELECT b.name as community,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data a
JOIN community_areas b
on a.community_area = b.community_area
GROUP BY community
ORDER BY 2 DESC'''),
x = 'community',
y = 'Count',
xlabel = 'Community Area',
ylabel = 'Service Request Count',
title = 'Service Requests by Community Area',
xsize = 9)

### Graffiti Removal Requests
Graffiti removal requests account for a large proportion of requests. Relatively recent gentrification in the area could explain both the high presenence and high reporting of graffiti. 

In [43]:
barplot(pq('''
SELECT b.name as community,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data a
JOIN community_areas b
on a.community_area = b.community_area
WHERE type_of_service_request = 'Graffiti Removal'
GROUP BY community
ORDER BY 2 DESC'''),
x = 'community',
y = 'Count',
xlabel = 'Community Area',
ylabel = 'Graffiti Service Request Count',
title = 'Graffiti Service Requests by Community Area',
xsize = 9)

### Vacant and Abandoned Building Requests
More than half of vacant and abandon building requests are on the South side of Chicago. A notable and self-explanatory exception is Hyde Park, which had only one reported vacant building, which stands in deep contrast to neighboring Woodlawn (45 reports) and Washington Park (16 reports), and Englewood (110 reports).

In [44]:
barplot(pq('''
SELECT b.name as community,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data a
JOIN community_areas b
on a.community_area = b.community_area
WHERE type_of_service_request = 'Vacant/Abandoned Building'
GROUP BY community
ORDER BY 2 DESC'''),
x = 'community',
y = 'Count',
xlabel = 'Community Area',
ylabel = 'Vacant/Abandoned Building Service Request Count',
title = 'Vacant/Abandoned Building Requests by Community Area',
xsize = 9)

### Alley Light Out Service Requests
Austin saw notably higher alley lights reported compared to other community areas.

In [39]:
barplot(pq('''
SELECT b.name as community,
COUNT(DISTINCT service_request_number) as Count
FROM combined_data a
JOIN community_areas b
on a.community_area = b.community_area
WHERE type_of_service_request = 'Alley Light Out'
GROUP BY community
ORDER BY 2 DESC'''),
x = 'community',
y = 'Count',
xlabel = 'Community Area',
ylabel = 'Alley Light Out Service Request Count',
title = 'Alley Light Out Service Requests by Community Area',
xsize = 8)

In [4]:
pq('''
with temp as (
SELECT substr(fips,6,6) as tract,
block_group,
COUNT(DISTINCT service_request_number) as Count,
avg(days_to_complete) as days
FROM combined_data
WHERE type_of_service_request != 'Graffiti Removal'
GROUP BY tract, block_group)
SELECT temp.*, 
cb.*, 
cb.white*1.0/total_population as pct_white,
cb.black*1.0/total_population as pct_black,
cb.asian*1.0/total_population as pct_asian
FROM temp
JOIN census_block cb
ON temp.tract = cb.tract
AND temp.block_group = cb.block_group
''')


,tract,block_group,Count,days,total_population,white,black,asian,hispanic,poverty,...,total_employed,unemployed,median_home_value,block_group,county,state,tract,pct_white,pct_black,pct_asian
0,010100,1,1,203.000000,639,288,232,0,,,...,555,44,502500,1,031,17,010100,0.450704,0.363067,0.000000
1,010201,1,1,165.000000,1417,588,656,100,,,...,1209,101,161100,1,031,17,010201,0.414961,0.462950,0.070572
2,010201,2,7,67.714286,4641,1962,2319,82,,,...,3620,248,157200,2,031,17,010201,0.422754,0.499677,0.017669
3,010202,1,3,58.333333,1114,474,324,167,,,...,951,0,-666666666,1,031,17,010202,0.425494,0.290844,0.149910
4,010202,2,1,162.000000,1161,623,411,29,,,...,978,75,156800,2,031,17,010202,0.536606,0.354005,0.024978
5,010300,3,1,79.000000,1291,833,391,44,,,...,1115,37,168100,3,031,17,010300,0.645236,0.302866,0.034082
6,010300,4,2,169.500000,1854,1020,353,424,,,...,1359,19,227900,4,031,17,010300,0.550162,0.190399,0.228695
7,010400,2,1,73.000000,1088,572,344,128,,,...,847,68,125000,2,031,17,010400,0.525735,0.316176,0.117647
8,010400,3,2,172.000000,2802,2123,419,140,,,...,2688,154,314300,3,031,17,010400,0.757673,0.149536,0.049964
9,010502,1,6,32.000000,1174,694,244,64,,,...,1043,80,172200,1,031,17,010502,0.591141,0.207836,0.054514


## 311 Data Augmented With 2016 American Community Survey Data

In [12]:
r = pq('''
with temp as (
SELECT substr(fips,6,6) as tract,
block_group,
COUNT(DISTINCT service_request_number) as Count,
avg(days_to_complete) as days
FROM combined_data
WHERE type_of_service_request != 'Graffiti Removal'
or type_of_service_request = 'Alley Light Out'
GROUP BY tract, block_group)
SELECT temp.*, 
cb.*, 
cb.white*1.0/total_population as pct_white,
cb.black*1.0/total_population as pct_black,
cb.asian*1.0/total_population as pct_asian
FROM temp
JOIN census_block cb
ON temp.tract = cb.tract
AND temp.block_group = cb.block_group
''')

# Create a trace
trace = go.Scatter(
    x = list(r['pct_black']),
    y = list(r['days']),
    mode = 'markers'
)

data = [trace]
layout = go.Layout(
    title='<b>Service Requests by Percent Black</b><br>',
             xaxis=dict(
                title= 'Percent Black',
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
             yaxis=dict(
                title='Number of Requests',
                titlefont=dict(
                    size=16,
                    color='rgb(107, 107, 107)'
                ),
                tickfont=dict(
                    size=14,
                    color='rgb(107, 107, 107)'
                )
             ),
            )


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-scatter')
